## Criação de uma rede neural convulacional

Como o tensorflow utiliza o keras como backend, será necessário importá-las

In [1]:
import tensorflow as tf
print("Versão do TensorFlow:", tf.__version__)

Versão do TensorFlow: 2.3.0


In [3]:
import keras as K
print("Versão do Keras:", K.__version__)

Versão do Keras: 2.4.3


In [4]:
# Imports
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

In [5]:
# Inicializando a Rede Neural Convolucional
classifier = Sequential()

Depois de inicializada a rede neural, o shape dos dados de entrada e função de ativação precisam receber alguns paprâmetros. Será 32 feature para um array 2D e terá o formato 3x3.

In [6]:
# Passo 1 - Adicionando a primeira Camada de Convolução no objeto inicializado.
# Neste caso, o 'relu' significa uma função retificadora
# os 3 parâmetros são: quantidade de filtros, formato e formato dos dados de entrada (input_shape) - o 3 nesse parâmetro é oq define que a imagem de entrada será colorida.
classifier.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))

Agora precisamos, basicamente, diminuir a quantidade de nós nesta camada e ser passado pra próxima. Usaremos o agrupamento (pooling) para essa redução.

In [ ]:
# Passo 2 - Pooling (Os pixels foram agrupados em um conjunto de 2x2)
classifier.add(MaxPooling2D(pool_size = (2, 2)))

In [ ]:
# Adicionando a Segunda Camada de Convolução
classifier.add(Conv2D(32, (3, 3), activation = 'relu'))


In [ ]:
# pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))

In [ ]:
# Passo 3 - Flattening - Achatamento
# Estamos convertendo a estrutura do dado 2D em uma estrutura de 1D
#Não precisamos adicionar nenhum parâmetro por que o Keras já entende que objeto já esteja "segurando" os pixels da imagem e eles precisam ser nivelados.
classifier.add(Flatten())

Nesta camada precisamos conectar todas as camadas.
Após gerado alguns nós após a etapa de nivelamento, esses nós serão conectados a esta camada. Esta camada pode ser chamada de camada oculta, uma vez que ela está entre a camada de entrada e a camada de saída.

In [ ]:
# Usamos uma função de ativação retificadora (relu) e então uma função de ativação sigmóide para obter as probabilidades de cada imagem conter um cachorro ou um gato.
# Passo 4 - Full connection
classifier.add(Dense(units = 128, activation = 'relu'))

# Agora é hora de inicializar nossa camada de saída, que deve conter apenas um nó, pois é uma classificação binária. Este único nó nos dará uma saída binária de um gato ou cachorro.
classifier.add(Dense(units = 1, activation = 'sigmoid'))

Como você pode ver, Dense é a função para adicionar uma camada totalmente conectada, 'unidades' é onde definimos o número de nós que devem estar presentes nesta camada oculta, o valor dessas unidades estará sempre entre o número de nós de entrada e o nós de saída, mas a arte de escolher o número ideal de nós só pode ser alcançada por meio de tentativas experimentais. Embora seja uma prática comum usar uma potência de 2. E a função de ativação será uma função retificadora.

In [ ]:
# Agora que foi concluido a construção do modelo CNN, é hora de compilar.
# Compilando a rede
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

Para essa compilação foi escolhido o otimizador 'adam'é um algoritmo de primeira ordem para otimização baseado em gradiente de funções objetivas estocásticas, que toma como base uma estimativa adaptada de momentos de baixa ordem.

## Treinando a Rede Neural Convulacional

Para a tarefa de treinamento, vamos usar a função ImageDataGenerator() do Keras e ajustar escala e zoom das imagens de treino e a escala das imagens de validação.

In [ ]:
tion_datagen com as regras de pré-processamento das imagens
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

validation_datagen = ImageDataGenerator(rescale = 1./255)

Explanação desses parâmetros tem na documentação do Keras, mas resumidamente, é criado dados sintéticos a partir das mesmas imagens, realizando diferentes tipos de operações nessas imagens, como virar, girar, desfocar etc.

Aplicamos então os dois objetos criados anteriormente para pré-processar os dados de treino e de validação. Lembre-se: o tratamento aplicado aos dados de validação deve ser o mesmo tratamento aplicado aos dados de treino.

In [ ]:
validação
training_set = train_datagen.flow_from_directory('dataset_treino',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

validation_set = validation_datagen.flow_from_directory('dataset_validation',
                                                        target_size = (64, 64),
                                                        batch_size = 32,
                                                        class_mode = 'binary')

# Treinamento